# This Notebook is for data processing development

Here we will calculate the model parameters

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./../data/database.csv', sep=';')

In [3]:
data.head()

,ID_PROBLEMA,ID_PACIENTE,ID PACIENTE,ID MÉDICO,ID_LUGAR,CENTRO,FECHA_EVOLUCION,ETIQUETA_GES,ID_AGRUPADOR,POTENCIAL_1,POTENCIAL_2,POTENCIAL_3
0,17648485,211402,1399,1,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,POSIBLE,59.0,56.0,NaN,NaN
1,17644905,5845159,1399,1,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,NO GES,77.0,56.0,NaN,NaN
2,17730364,10199783,1399,1,21.0,CMD PEDRO DE VALDIVIA,29/07/2019,POSIBLE,56.0,NaN,NaN,NaN
3,17677869,10824784,5268,1,21.0,CMD PEDRO DE VALDIVIA,02/08/2019,NaN,NaN,76.0,NaN,NaN
4,17656698,3767945,5751,1,21.0,CMD PEDRO DE VALDIVIA,09/08/2019,NO GES,56.0,NaN,NaN,NaN


# Add id to medical centers

In [4]:
medical_centers = data.CENTRO.unique()

In [5]:
# fix string format
for i in range(len(medical_centers)):
    medical_centers[i] = medical_centers[i].replace('\xa0', ' ')
medical_centers

array(['CMD PEDRO DE VALDIVIA', 'CMD IQUIQUE', 'CMD ALAMEDA', 'CMD ÑUÑOA',
       'CMD LA FLORIDA', 'CMD VALDIVIA', 'CMD QUILICURA',
       'CMD VIÑA DEL MAR', 'CMD SAN BERNARDO', 'CMD MUELLE BARON',
       'CMD MAIPU', 'CMD CONCEPCION', 'CMD OSORNO', 'CMD SAN MIGUEL',
       'CMD ANTOFAGASTA', 'CMD RANCAGUA', 'CMD PUENTE ALTO',
       'CMD PUERTO MONTT', 'CMD TEMUCO CENTRO', 'CMD CHILLAN', 'CD ELQUI',
       'CMD PUNTA ARENAS', 'CMD CONCHALI', 'CD KENNEDY',
       'CMD LOS ANGELES', 'CMD QUILPUE', 'CMD CALAMA', 'CMD ARICA',
       'CMD TALCA', 'CD PUENTE ALTO', 'CD LA REINA', 'CD INDEPENDENCIA',
       'CD PASEO ESTACION', 'CD PEÑALOLEN'], dtype=object)

In [21]:
enumerated = list(enumerate(medical_centers, start=1))

In [22]:
for index, row in data.iterrows():
    for med in enumerated:
        if row['CENTRO'] == med[1]:
            data.loc[index, 'ID_LUGAR'] = med[0]

In [23]:
data.ID_LUGAR.unique()

array([ 1.,  2., 27., 25., 23., 17.,  7., 29., 10., 22., 13., 18., 14.,
        4., 24., 19., 21., 20., 28., 16.,  9.,  5., 12., 30., 31., 32.,
       33., 34.])

In [24]:
# data.to_csv(r'./new.csv')

In [25]:
data.head()

,ID_PROBLEMA,ID_PACIENTE,ID PACIENTE,ID MÉDICO,ID_LUGAR,CENTRO,FECHA_EVOLUCION,ETIQUETA_GES,ID_AGRUPADOR,POTENCIAL_1,POTENCIAL_2,POTENCIAL_3
0,17648485,211402,1399,1,1.0,CMD PEDRO DE VALDIVIA,29/07/2019,POSIBLE,59.0,56.0,NaN,NaN
1,17644905,5845159,1399,1,1.0,CMD PEDRO DE VALDIVIA,29/07/2019,NO GES,77.0,56.0,NaN,NaN
2,17730364,10199783,1399,1,1.0,CMD PEDRO DE VALDIVIA,29/07/2019,POSIBLE,56.0,NaN,NaN,NaN
3,17677869,10824784,5268,1,1.0,CMD PEDRO DE VALDIVIA,02/08/2019,NaN,NaN,76.0,NaN,NaN
4,17656698,3767945,5751,1,1.0,CMD PEDRO DE VALDIVIA,09/08/2019,NO GES,56.0,NaN,NaN,NaN
